# Neural Network - Iris data set :-

#### Title of the data set: Iris Plants Database

#### Number of instance: 150

#### Number of attributes : 5 (4 predictive, 1 name)

#### Attribute Information:

   1. sepal length in cm
   2. sepal width in cm
   3. petal length in cm
   4. petal width in cm
   5. **Class:** Species of the flower
              - Iris Setosa
              - Iris Versicolour
              - Iris Virginica  

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
pd.set_option('display.max_rows', 200)

In [3]:
df = pd.read_csv('../data/iris.data', sep=',', header=None, names=['SepalLengthCm','SepalWidthCm',
                                                                   'PetalLengthCm','PetalWidthCm','Species'])

In [4]:
df.head()

SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0            5.1           3.5            1.4           0.2  Iris-setosa
1            4.9           3.0            1.4           0.2  Iris-setosa
2            4.7           3.2            1.3           0.2  Iris-setosa
3            4.6           3.1            1.5           0.2  Iris-setosa
4            5.0           3.6            1.4           0.2  Iris-setosa

In [5]:
#Prepare the input data
X = df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
X = np.array(X)
X[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [6]:
#Using one-hot-encoding technique to map the categorical value of species to numerical i.e. 
#(Iris-setosa, Iris-versicolor, Iris-virginica) to (0,1,2) and then to one-hot encoded 
#([1, 0, 0], [0, 1, 0], [0, 0, 1])
one_hot_encoder = OneHotEncoder(sparse=False)

Y = df.Species
Y = one_hot_encoder.fit_transform(np.array(Y).reshape(-1, 1))
Y[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [7]:
#Split the data set into train/validation/test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1)

## Implementation of Neural Network

### Neural network consists of the following components:
        - An input layer, x
        - Arbitary number of hidden layers
        - An out put layer, y
        - A set of weights and biases between each layer, W and b
        - A choice of activation function for each hidden layer, σ (Sigmoid activation function used here)

### Initialize weights and biases !!

#### How it works ?
        - The weights of the network are initialized randomly in the range [-1,1]
        
        - The bias has a constant value of 1
        
        - The function takes nodes as input which is basically a list of integers denoting the number of nodes in 
        each layer and length of the list denotes the number of layers
        
        - The function returns a multi-dimensional array as output which corresnpond to the weights
        
        - Each element in the weights list represents a hidden layer and has the weights of connections from the 
        previous layer (including the bias) to the current layer.
        
        
        
The right values for the weights and biases determines the strength of the predictions. The process of fine-tuning the weights and biases from the input data is known as training the Neural Network.

In [8]:
def initializeWeight(nodes):
    layers, weights = len(nodes), []
    
    for i in range(1, layers):
        wt = [[np.random.uniform(-1, 1) for k in range(nodes[i-1] + 1)] for j in range(nodes[i])]
        weights.append(np.matrix(wt))
    
    return weights

### The main function !!

#### How it works ?

    - This function actually trains the network for given number of iterations. The input parameters are:
        - training data and target values
        - validation data and target values
        - number of iterations , default=10
        - list of integers
        - learning rate of back-propagation algorithm, default=0.15

In [9]:
def neuralNet(X_train, Y_train, X_val=None, Y_val=None, iterations=10, nodes=[], rate=0.15):
    hiddenLayers = len(nodes) - 1
    weights = initializeWeight(nodes)

    for iteration in range(1, iterations+1):
        weights = trainNetwork(X_train, Y_train, rate, weights)

        #Print the accuracy of training and validation after every 20 iterations
        if(iteration % 20 == 0):
            print("Iteration {}".format(iteration))
            print("Training Accuracy:{}".format(accuracy(X_train, Y_train, weights)))
            if X_val.any():
                print("Validation Accuracy:{}".format(accuracy(X_val, Y_val, weights)))
                        
    return weights

The weights need to be continuously adjusted across each iteration to increase the accuracy of the network. In each iteration the network is trained using forward/backward propagation algorithm. First, the input is passed through the network and output is calculated and then, according to the error of output, the weights are updated backwards. So basically, the error is propagated backward.

### Forward Propagation !!

    - Each layer receives an input and computes the output which is calculated by passing the dot product of input and weights of the layer to a activation function(Sigmiod in this case).
    
    - The output of each layer is the input for the next layer
    
    - The input of the first layer is feature vector and output of last layer is the prediction
    

In [10]:
def feedForward(x, weights, layers):
    output, current_input = [x], x
    for j in range(layers):
        activation = Sigmoid(np.dot(current_input, weights[j].T))
        output.append(activation)
        current_input = np.append(1, activation) # add the bias = 1
    
    return output

### Backward Propagation !!

    - Calculate the error rate of the final output
    
    - Propagate the error backwards and adjusts the weights
        - Delta is calculated as : error of next layer times sigmoid derivation of the current layer output
        
        - Weights between current and previous layer are updated as: multiple Delta with the putput of previous 
        layer and rate, then add this to the weight of previous layer
        
        - calculate error of the current layer, rmeove the bias form the weights of the previous layer and 
        then multipy this result with Delta to get error

In [11]:
def backPropagation(y, output, weights, layers):
    outputFinal = output[-1]
    error = np.matrix(y - outputFinal) #Calculate the error at last output
    
    #Back propagate the error
    for j in range(layers, 0, -1):
        currOutput = output[j]
        
        if(j > 1):
            # Add previous output
            prevOutput = np.append(1, output[j-1])
        else:
            prevOutput = output[0]
        
        delta = np.multiply(error, sigmoidDerivative(currOutput))
        weights[j-1] += rate * np.multiply(delta.T, prevOutput)

        wt = np.delete(weights[j-1], [0], axis=1) # Remove bias from weights
        error = np.dot(delta, wt) # Calculate error for current layer
    
    return weights

### Training and Prediction functions !!

In [12]:
#This will perform forward and backward propagation, the new weights will be returned n the end
def trainNetwork(X, Y, rate, weights):
    layers = len(weights)
    for i in range(len(X)):
        x, y = X[i], Y[i]
        x = np.matrix(np.append(1, x)) # Add feature vector
        
        output = feedForward(x, weights, layers)
        weights = backPropagation(y, output, weights, layers)

    return weights

### Activation Function !!

    - Activation function of a node defines the output of that node, or "neuron," given an input or set of inputs 
    
    - This output is then used as input for the next node and so on until a desired solution to the original 
    problem is found
    
    - It maps the resulting values into the desired range such as between 0 to 1 or -1 to 1 etc.
    
    - Activation function gets to decide which neurons will push forward the values into the next layer
    
    - Sigmoid activation function has been used in this assignment.
     It takes a value as input and outputs another value between 0 and 1. It is non-linear and easy to work 
     with when constructing a neural network model. 

In [13]:
def Sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoidDerivative(x):
    return np.multiply(x, 1-x)

### How does the prediction work ?

    - The input is first passed to the network
    - The output will be array of three real numbers corresponding to the three species
    - The higher value of a number indicates the most probable class
    - findMaxActivation() will find the maximum valued output and then corresponding index is set to 1
    - So, the predicted class is the one that the network is most confident of

In [14]:
def predict(item, weights):
    layers = len(weights)
    item = np.append(1, item)
    
    #forward propagation
    output = feedForward(item, weights, layers)
    
    outputFinal = output[-1].A1
    index = findMaxActivation(outputFinal)

    y = [0 for i in range(len(outputFinal))]
    y[index] = 1

    return y

In [15]:
def findMaxActivation(output):
    m, index = output[0], 0
    for i in range(1, len(output)):
        if(output[i] > m):
            m, index = output[i], i
    
    return index

### Check the accuracy of prediction !!

    - given the computed weights, the model predicts the class of each object in its input
    - predicted class is checked against the actual class
    - increase the number of correct classfication if it matches
    - percentage of correct predictions is returned

In [16]:
def accuracy(X, Y, weights):
    correct_classification = 0

    for i in range(len(X)):
        x, y = X[i], list(Y[i])
        prediction = predict(x, weights)

        if(y == prediction):
            correct_classification += 1

    return correct_classification / len(X)

In [17]:
# Run it here
features = len(X[0]) # Number of features - using all of them
classes = len(Y[0]) # Number of classes

layers = [features, 5, 10, classes]
rate, iterations = 0.15, 100

weights = neuralNet(X_train, Y_train, X_val, Y_val, iterations=iterations, nodes=layers, rate=rate)

Iteration 20
Training Accuracy:0.8947368421052632
Validation Accuracy:0.9230769230769231
Iteration 40
Training Accuracy:0.8859649122807017
Validation Accuracy:0.9230769230769231
Iteration 60
Training Accuracy:0.8508771929824561
Validation Accuracy:0.9230769230769231
Iteration 80
Training Accuracy:0.8508771929824561
Validation Accuracy:0.9230769230769231
Iteration 100
Training Accuracy:0.9736842105263158
Validation Accuracy:1.0


In [18]:
#Final testing accuracy
print("Testing Accuracy: {}".format(accuracy(X_test, Y_test, weights)))

Testing Accuracy: 0.8695652173913043


### Check an example prediction !!

In [19]:
print(X_test[0], list(Y_test[0]))

[6.4 3.1 5.5 1.8] [0.0, 0.0, 1.0]


In [20]:
print(predict(X_test[0], weights))

[0, 0, 1]


In [21]:
#Predicts the same class

In [ ]:
#Reference: 
#https://towardsdatascience.com/how-to-build-your-own-neural-network-from-scratch-in-python-68998a08e4f6
#https://towardsdatascience.com/neural-network-on-iris-data-4e99601a42c8